<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/modelToDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##モデルをDB化

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [10]:
import IPython
#bz2ファイル解凍
#!bunzip2 -k -q "./drive/My Drive/NLP/20170201.tar.bz2"

#tarファイル解凍
!tar -xvf "./drive/My Drive/NLP/20170201.tar"

entity_vector/
entity_vector/entity_vector.model.txt
entity_vector/entity_vector.model.bin


In [12]:
from gensim.models import KeyedVectors

#解凍時のみ
#model_dir = 'entity_vector/entity_vector.model.bin'
#model = KeyedVectors.load_word2vec_format(model_dir, binary=True,limit=500000)

#解凍後確認
results = model.most_similar(positive=[u'数学'])
for result in results:
  print(result)

('[数学]', 0.8441122770309448)
('物理学', 0.8005046844482422)
('算術', 0.7927638292312622)
('[幾何学]', 0.7776163220405579)
('[物理学]', 0.7547159194946289)
('[代数学]', 0.7514173984527588)
('代数学', 0.7450574636459351)
('[解析学]', 0.741286039352417)
('[確率論]', 0.7390509843826294)
('[数論]', 0.7337667942047119)


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [13]:
!pip install mecab-python3

     |████████████████████████████████| 15.9MB 2.7MB/s 


In [29]:
import re
import pandas as pd

cnt = 1
model_vocab_list = []

for words, v in model.vocab.items():

  words = words.replace('_', '')
  
  #固有名詞用の括弧を削除
  words = words.replace('[', '').replace(']', '')

  #補足情報の_を削除
  words = words.replace("_","")
  #括弧文字を抽出
  regex = re.compile(".*?\((.*?)\)")
  #括弧文字をlist型で返却
  ret_list = re.findall(regex, words)

  #括弧文字がある場合は無効化
  if len(ret_list) > 0: 
    words = words.replace("(" + ret_list[0] + ")",'')
    model_vocab_list.append(words)
  else:
    model_vocab_list.append(words)
    
  if cnt % 50000 == 0:
    print(cnt,"件 終了");
    
  cnt = cnt + 1

#重複削除
model_vocab_uni_list= list(set(model_vocab_list))
  
#保存
df = pd.DataFrame(model_vocab_uni_list)
df.to_csv('./drive/My Drive/NLP/model_vocab_list.csv', header=False, index=False)

50000 件 終了
100000 件 終了
150000 件 終了
200000 件 終了
250000 件 終了
300000 件 終了
350000 件 終了
400000 件 終了
450000 件 終了
500000 件 終了


In [30]:
csv_dir = './drive/My Drive/NLP/model_vocab_list.csv'
df = pd.read_csv(csv_dir,names=['words'])

print(df.count())
print(df[df["words"]=="ゆず"])

words    425495
dtype: int64
       words
309355    ゆず


In [0]:
import re
import pandas as pd

#対義語生成
import lib_wordRevChange as lw

cnt = 1
save_list = []

for words, v in model.vocab.items():

  words = words.replace('_', '')
  
  #固有名詞用の括弧を削除
  words = words.replace('[', '').replace(']', '')

  #補足情報の_を削除
  words = words.replace("_","")
  #括弧文字を抽出
  regex = re.compile(".*?\((.*?)\)")
  #括弧文字をlist型で返却
  ret_list = re.findall(regex, words)
  #括弧文字がある場合は無効化
  if len(ret_list) > 0: 
    words = words.replace("(" + ret_list[0] + ")",'')
  
    gyaku = u"逆"
    inherent_words = '[' + words + ']'  
    rev_list = lw.wordRevChange(words,gyaku,inherent_words,model)
    save_list.append(rev_list)

  if cnt % 500 == 0:
    print(cnt,"件 終了");
    
  cnt = cnt + 1

#保存
df = pd.DataFrame(save_list)
df.to_csv('./drive/My Drive/NLP/modelToCsv.csv', header=False, index=False)

In [8]:
import re

mystring = "あいうえお_(補足情報)"
mystring = mystring.replace("_","")

regex = re.compile(".*?\((.*?)\)")

#list型で返却
result = re.findall(regex, mystring)

ret_word = mystring.replace("(" + result[0] + ")",'')

print(ret_word)

あいうえお
